<h1 align="center">Web Mining Lab Assignment-13
</h1>
                                                                                                                                        <h3 align="right">Aryan Vigyat</h3>
                                                                                                            <h3 align="right"> 20BCE1452</h3>

Use the same program to find do sessionization of a log file that is downloaded from web that is uploaded into moodle today and give it in same format as Q1.

In [1]:
# Implementing sessionization on log entry dataset
# Importing the libraries
from csv import reader
from datetime import datetime

In [2]:
class Sessionize(object):
	def __init__(self, filename, delta):
		"""Function to initialize the different parameters of the object."""
		self.delta = delta
		csvfile = open(filename, 'r')
		self.dataset_ = list(reader(csvfile))
		## Removing the redundant IP addresses
		classes = list(set(row[0] for row in self.dataset_))
		classes = sorted(classes)
		temp = []
		for row in self.dataset_:
			if row[0] in classes[:5]:
				temp.append(row)
		self.dataset_ = temp
		self.updateOrderingOfEntries()

	def separateUsers(self):
		"""Function to separate the server log entries based on the user 
		i.e. on the basis of the IP and user-agent."""
		self.separate_ = {}
		for row in self.dataset_:
			if row[0] not in self.separate_:
				self.separate_[row[0]] = []
			self.separate_[row[0]].append(row[1:])
		# Updating the timestamp field of the entries.
		self.updateTimestamp()

	def updateTimestamp(self):
		"""Function that updates the timestamp field in a format that 
		makes its processing by datetime module easy."""
		for i in self.separate_:
			for j in self.separate_[i]:
				# Trimming unnecessary characters from the entries
				date_time = j[0][1:]
				j[0] = date_time
				# print(date_time)
	
	# def updateOrderingOfEntries(self):
	# 	"""Function to sort the entries in ascending order based on the 
	# 	timestamp using Bubble Sort."""

	# 	for i in range(len(self.dataset_) - 1):
	# 		for j in range(i + 1, len(self.dataset_)):
	# 			t1 = datetime.strptime(self.dataset_[i][1][1:],
	# 				 "%d/%b/%Y:%H:%M:%S")
	# 			t2 = datetime.strptime(self.dataset_[j][1][1:],
	# 				 "%d/%b/%Y:%H:%M:%S")
	# 			if t1 > t2:
	# 				self.dataset_[i], self.dataset_[j] = self.dataset_[j], self.dataset_[i]

	def updateOrderingOfEntries(self):
		"""Function to sort the entries in ascending order based on the 
		timestamp using Selection Sort."""

		for i in range(len(self.dataset_)):
			# Find the minimum element in the remaining unsorted array.
			min_idx = i
			t1 = datetime.strptime(self.dataset_[i][1][1:], 
					 "%d/%b/%Y:%H:%M:%S")
			for j in range(i+1, len(self.dataset_)):
				t2 = datetime.strptime(self.dataset_[j][1][1:], 
					 "%d/%b/%Y:%H:%M:%S")
				if t1 > t2:
					min_idx = j

			# Swapping the minimum element at ith index
			self.dataset_[i], self.dataset_[min_idx] = self.dataset_[min_idx], self.dataset_[i]

	def createSession(self):
		"""Function to create session for each user based on the different
		rules of sessionization."""
		self.sessions_ = {}
		for i in self.separate_:
			if i not in self.sessions_:
				self.sessions_[i] = []
			for j in range(len(self.separate_[i])):
				temp = []
				present = False
				for l in self.sessions_[i]:
					if self.separate_[i][j] in l:
						present = True
				if not present:
					temp.append(self.separate_[i][j])
					for k in range(j + 1, len(self.separate_[i])):
						t1 = datetime.strptime(self.separate_[i][j][0], 
							"%d/%b/%Y:%H:%M:%S")
						t2 = datetime.strptime(self.separate_[i][k][0], 
							"%d/%b/%Y:%H:%M:%S")
						latest = max((t1, t2))
						old = min((t1, t2))
						difference = latest - old
						if(difference.seconds <= self.delta):
							temp.append(self.separate_[i][k])
					self.sessions_[i].append(temp)

	def printSessions(self):
		"""Function to print the sessions per user."""
		session_id = 1
		print('%s' % ('-' * 93))
		print('| {:^20} | {:^20} | {:^20} | {:^20} |'.format("Session Id",
			"IP address", "Start Time", "End Time"))
		print('%s' % ('-' * 93))
		for i in self.sessions_:
			for l in self.sessions_[i]:
				dates = []
				for row in l:
					dates.append(datetime.strptime(row[0], 
							"%d/%b/%Y:%H:%M:%S"))
				print('| {:^20} | {:^20} | {:^20} | {:^20} |'.format(session_id, i, str(min(dates)), str(max(dates))))
				session_id += 1
		print('%s' % ('-' * 93))

In [3]:
## Main code
filename = input('Enter the name of the dataset: ')
delta = int(input('Enter delta value (minutes): '))
# Coverting the delta form minutes to seconds.
delta *= 60
# Creating the Sessionize object and calling appropriate functions.
session_create = Sessionize(filename, delta)
session_create.separateUsers()
session_create.createSession()
session_create.printSessions()

Enter the name of the dataset: /content/drive/MyDrive/weblog.csv
Enter delta value (minutes): 30
---------------------------------------------------------------------------------------------
|      Session Id      |      IP address      |      Start Time      |       End Time       |
---------------------------------------------------------------------------------------------
|          1           |      10.131.2.1      | 2017-11-09 22:16:22  | 2017-12-01 22:21:14  |
|          2           |      10.131.2.1      | 2017-11-18 06:36:59  | 2017-12-01 07:04:32  |
|          3           |      10.131.2.1      | 2017-11-12 13:11:42  | 2017-11-30 14:05:10  |
|          4           |      10.131.2.1      | 2017-11-23 14:21:45  | 2017-12-03 14:51:34  |
|          5           |      10.131.2.1      | 2017-11-09 14:36:26  | 2017-12-02 15:27:04  |
|          6           |      10.131.2.1      | 2017-11-09 15:16:36  | 2017-12-03 15:53:50  |
|          7           |      10.131.2.1      | 2017-11-0